# Interactive Brokers Connection Example

This notebook demonstrates basic connection and operations with Interactive Brokers using the ib_async library.

## Connection

**Important Note**: From now on, use `ib_async` instead of the older Interactive Brokers API.

In [4]:
from ib_async import *
util.startLoop()  # required in Jupyter (interactive environments), not in scripts

In [5]:
ib = IB()

**Make sure you are logged in your TWS!!**

In [6]:
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [8]:
ib.positions()

[]

In [9]:
ib.disconnect()

'Disconnecting from 127.0.0.1:7497, 117 B sent in 8 messages, 18.9 kB received in 398 messages, session time 5.85 s.'

## Contracts (Introduction)

Working with contracts is fundamental in Interactive Brokers. A contract represents a financial instrument and contains all the information needed to uniquely identify it on the exchange.

Here we'll create and qualify a Forex contract. The `qualifyContracts()` function fills in missing contract details by querying Interactive Brokers' database.

In [10]:
from ib_async import *
util.startLoop()

In [11]:
ib = IB()

In [12]:
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [13]:
contract = Forex("EURUSD")
contract

Forex('EURUSD', exchange='IDEALPRO')

Create a Forex contract for EUR/USD. The `Forex()` constructor creates a contract object, but it's not yet validated with IB:

In [14]:
contract = ib.qualifyContracts(contract)
contract

[Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')]

The `qualifyContracts()` function validates the contract with IB and fills in missing details like contract ID, exchange info, etc.:

In [15]:
type(contract)

list

Check the type - `qualifyContracts()` returns a list, even for a single contract:

In [16]:
contract[0]

Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')

Access the first (and only) contract in the returned list to see the full contract details:

In [17]:
contract[0].conId

12087792

The `conId` (contract ID) is a unique identifier assigned by Interactive Brokers to each financial instrument:

## Contract Types

Interactive Brokers supports various contract types. Each type has specific parameters:

- **Contract(conId)**: Direct reference using IB's contract ID
- **Stock**: Equity securities (symbol, exchange, currency, primaryExchange optional)
- **Forex**: Currency pairs (currency pair like 'EURUSD')
- **CFD**: Contracts for Difference (symbol)
- **Future**: Futures contracts (symbol, expiry date, exchange)
- **Option**: Options contracts (symbol, expiry, strike, right, exchange)
- **Bond**: Fixed income securities (using ISIN or other identifiers)

In [18]:
# Contract(conId=270639)
# Stock('AMD', 'SMART', 'USD')
# Stock('INTC', 'SMART', 'USD', primaryExchange='NASDAQ')
# Forex('EURUSD')
# CFD('IBUS30')
# Future('ES', '20180921', 'GLOBEX')
# Option('SPY', '20170721', 240, 'C', 'SMART')
# Bond(secIdType='ISIN', secId='US03076KAA60')

### Handling Ambiguous vs Unambiguous Contracts

When creating contracts, some may be **ambiguous** (multiple matches) while others are **unambiguous** (single match):

- **Unambiguous**: When you provide enough details (symbol + exchange + currency), IB can uniquely identify the contract
- **Ambiguous**: When details are incomplete (e.g., just symbol), IB may find multiple matching contracts

The `qualifyContracts()` function behaves differently in each case:

In [19]:
contract = Stock("AAPL", "SMART", "USD")
contract

Stock(symbol='AAPL', exchange='SMART', currency='USD')

**Unambiguous case**: Full contract details provided (symbol, exchange, currency):

In [20]:
contract = ib.qualifyContracts(contract)  # unambiguous
contract

[Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS')]

This should return a single qualified contract since the details uniquely identify AAPL:

In [21]:
contract = ib.qualifyContracts(Stock("AAPL"))  # ambiguous
contract

Ambiguous contract: Stock(symbol='AAPL'), possibles are [Contract(secType='STK', conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), Contract(secType='STK', conId=273982664, symbol='AAPL', exchange='SMART', primaryExchange='EBS', currency='CHF', localSymbol='AAPL', tradingClass='AAPL'), Contract(secType='STK', conId=532640894, symbol='AAPL', exchange='SMART', primaryExchange='TSE', currency='CAD', localSymbol='AAPL', tradingClass='AAPL'), Contract(secType='STK', conId=265598, symbol='AAPL', exchange='AMEX', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), Contract(secType='STK', conId=265598, symbol='AAPL', exchange='NYSE', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), Contract(secType='STK', conId=265598, symbol='AAPL', exchange='CBOE', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), Contract(secType='ST

[None]

**Ambiguous case**: Only symbol provided, no exchange or currency specified. This may return multiple contracts if AAPL trades on different exchanges or in different currencies:

In [22]:
ib.disconnect()

'Disconnecting from 127.0.0.1:7497, 250 B sent in 11 messages, 49.6 kB received in 434 messages, session time 36.8 s.'

Finally, disconnect from Interactive Brokers when done:

# Current Market Data

This section demonstrates how to retrieve real-time market data for financial instruments using Interactive Brokers.

In [23]:
import pandas as pd
from ib_async import *
util.startLoop()

In [24]:
ib = IB()

In [26]:
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [27]:
contract = Forex("EURUSD")
contract = ib.qualifyContracts(contract)[0]  # Qualify and get first result
contract

Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')

In [28]:
data1 = ib.reqMktData(contract)
data1

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), defaults=IBDefaults(emptyPrice=-1, emptySize=0, unset=nan, timezone=datetime.timezone.utc), created=True)

In [29]:
data1

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2025, 11, 6, 20, 7, 45, 424804, tzinfo=datetime.timezone.utc), timestamp=1762459665.4248161, minTick=1e-05, bid=1.15462, bidSize=7500000.0, ask=1.15464, askSize=1500000.0, last=1.15465, lastSize=0.0, lastTimestamp=datetime.datetime(2025, 11, 6, 20, 6, 50, tzinfo=datetime.timezone.utc), prevBid=1.15464, prevBidSize=5000000.0, prevAsk=1.15465, prevAskSize=9000000.0, volume=0.0, high=1.15525, low=1.1491, close=1.1493, ticks=[TickData(time=datetime.datetime(2025, 11, 6, 20, 7, 45, 424804, tzinfo=datetime.timezone.utc), tickType=1, price=1.15462, size=7500000.0), TickData(time=datetime.datetime(2025, 11, 6, 20, 7, 45, 424804, tzinfo=datetime.timezone.utc), tickType=2, price=1.15464, size=1500000.0)], defaults=IBDefaults(emptyPrice=-1, emptySize=0, unset=nan, timezone=datetime.timezone.utc), created=True)

In [30]:
data1.ask

1.15464

In [31]:
data1.askSize

1500000.0

In [32]:
data1.marketPrice()

1.15463

In [33]:
data1.time

datetime.datetime(2025, 11, 6, 20, 7, 52, 478927, tzinfo=datetime.timezone.utc)

In [34]:
data1.time

datetime.datetime(2025, 11, 6, 20, 7, 54, 380514, tzinfo=datetime.timezone.utc)

In [35]:
pd.to_datetime(data1.time)

Timestamp('2025-11-06 20:07:55.543921+0000', tz='UTC')

In [36]:
contract = Stock("AAPL", "SMART", "USD")
contract

Stock(symbol='AAPL', exchange='SMART', currency='USD')

In [37]:
contract = Forex("EURUSD")
contract = ib.qualifyContracts(contract)[0]  # Qualify and get first result
contract
data2 = ib.reqMktData(contract)
data2

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2025, 11, 6, 20, 7, 59, 818951, tzinfo=datetime.timezone.utc), timestamp=1762459679.818957, minTick=1e-05, bid=1.15462, bidSize=10000000.0, ask=1.15464, askSize=5000000.0, last=1.15465, lastSize=0.0, lastTimestamp=datetime.datetime(2025, 11, 6, 20, 6, 50, tzinfo=datetime.timezone.utc), prevBid=1.15464, prevBidSize=9000000.0, prevAsk=1.15465, prevAskSize=5500000.0, volume=0.0, high=1.15525, low=1.1491, close=1.1493, defaults=IBDefaults(emptyPrice=-1, emptySize=0, unset=nan, timezone=datetime.timezone.utc), created=True)